In [1]:
#Already installed in the colab no need to install
#!pip3 install torch



[Reference Article Theory](https://colab.research.google.com/drive/109gHWFUlUzuwhgXROpzIuVoSPZA_qeoy)

In [0]:
import torch
import torch.nn as nn


In [0]:
# x is [numbers of hours studied , numbers of hrs slept]
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor

# y is the grade obtained by the student
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor

#test value case
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

In [4]:
print(X.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


In [0]:
#scaling the value based on the range of (max tensor value,0)
 # scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

#then dividing all the values to the max value for normalizing the data
X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100



## Model (Computation Graph)
Once the data has been processed and it is in the proper format, all you need to do now is to define your model. Here is where things begin to change a little as compared to how you would build your neural networks using, say, something like Keras or Tensorflow. However, you will realize quickly as you go along that PyTorch doesn't differ much from other deep learning tools. At the end of the day we are constructing a computation graph, which is used to dictate how data should flow and what type of operations are performed on this information. 

For illustration purposes, we are building the following neural network or computation graph:


![alt text](https://drive.google.com/uc?export=view&id=1l-sKpcCJCEUJV1BlAqcVAvLXLpYCInV6)

In [0]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

In [6]:

NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    if (i % 100) == 0:
        print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

print("Finished training!")

#0 Loss: 8721.85546875
#100 Loss: 8608.6669921875
#200 Loss: 8608.6669921875
#300 Loss: 8608.6669921875
#400 Loss: 8608.6669921875
#500 Loss: 8608.6669921875
#600 Loss: 8608.6669921875
#700 Loss: 8608.6669921875
#800 Loss: 8608.6669921875
#900 Loss: 8608.6669921875
Predicted data based on trained weights: 
Input (scaled): 
tensor([4., 8.])
Output: 
tensor([1.])
Finished training!


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Neural_Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
